In [1]:
# %%
import xarray as xr
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.stats import mannwhitneyu


In [11]:

# ----------------------------------------
# 📦 FUNCTIONS FOR LOADING, CLEANING, CLUSTERING, PLOTTING
# ----------------------------------------

def load_all_variables(nc_dir):
    """Load NetCDF files into a combined dataframe"""
    nc_paths = [os.path.join(dp, f) for dp, _, files in os.walk(nc_dir) for f in files if f.endswith(".nc")]
    ds = xr.open_mfdataset(nc_paths, combine='by_coords')
    df = ds.to_dataframe().reset_index()
    df['time'] = pd.to_datetime(df['time'], unit='s', origin='unix').dt.tz_localize("UTC").dt.tz_convert("America/Chicago")
    return df

def add_temporal_columns(df):
    """Add calendar-based time features"""
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    df['month'] = df['time'].dt.month
    df['weekday'] = df['time'].dt.weekday
    df['is_weekend'] = df['weekday'] >= 5
    return df

def assign_season(df):
    """Add a season label based on month"""
    df['season'] = 'Other'
    df.loc[df['month'].isin([12, 1, 2]), 'season'] = 'Winter'
    df.loc[df['month'].isin([3, 4, 5]), 'season'] = 'Spring'
    df.loc[df['month'].isin([6, 7, 8]), 'season'] = 'Summer'
    df.loc[df['month'].isin([9, 10, 11]), 'season'] = 'Fall'
    return df

def clean_by_qc_flags(df, qc_map):
    """Mask variables based on quality control flags"""
    dfc = df.copy()
    for var, qc_var in qc_map.items():
        if var in dfc.columns and qc_var in dfc.columns:
            dfc.loc[dfc[qc_var] > 1, var] = np.nan
    return dfc

def compute_stability(df, ustar_thresh=0.2, z=10.0):
    """Compute dimensionless stability z/L"""
    dfc = df[df['u*'] >= ustar_thresh].copy()
    dfc['inv_L'] = 1.0 / dfc['L']
    dfc.loc[~np.isfinite(dfc['inv_L']), 'inv_L'] = np.nan
    dfc['z_over_L'] = z * dfc['inv_L']
    return dfc

def remove_outliers_iqr(df, variables, k=1.5):
    """Remove outliers from specified columns using IQR"""
    dfc = df.copy()
    for var in variables:
        Q1 = dfc[var].quantile(0.25)
        Q3 = dfc[var].quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - k * IQR, Q3 + k * IQR
        dfc.loc[(dfc[var] < lower) | (dfc[var] > upper), var] = np.nan
    return dfc

def run_clustering_and_analysis(df, cluster_vars, nclust=5):
    """Main clustering + visualization pipeline"""
    df = df.copy()

    # Prepare wind components
    df['wind_u'] = df['u*'] * np.sin(np.radians(df['wind_dir']))
    df['wind_v'] = df['u*'] * np.cos(np.radians(df['wind_dir']))

    # Drop NA rows for clustering
    df = df.dropna(subset=cluster_vars)

    # Standardize and cluster
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cluster_vars])
    kmeans = KMeans(n_clusters=nclust, random_state=42, n_init='auto')
    df['regime'] = (kmeans.fit_predict(X_scaled) + 1).astype(str)

    cluster_ids = sorted(df['regime'].unique(), key=lambda x: int(x))
    palette = sns.color_palette("Set2", nclust)
    cluster_palette = {cid: palette[int(cid)-1] for cid in cluster_ids}

    # ------------------------------
    # 1️⃣ Thermodynamic & turbulence structure (2x1 panel)
    # ------------------------------
    fig, axes = plt.subplots(3, 1, figsize=(7, 10))
    sns.scatterplot(data=df, x='z_over_L', y='u*', hue='regime', palette=cluster_palette, ax=axes[0], s=30)
    axes[0].axvline(0, color='gray', linestyle='--')
    axes[0].set_title('Turbulence Structure: u* vs z/L')
    axes[0].set_xlabel('z / L'); axes[0].set_ylabel('u* [m/s]'); axes[0].grid(True)

    sns.scatterplot(data=df, x='z_over_L', y='air_temperature', hue='regime', palette=cluster_palette, ax=axes[1], s=30)
    axes[1].axvline(0, color='gray', linestyle='--')
    axes[1].set_title('Thermodynamic Structure: Temperature vs z/L')
    axes[1].set_xlabel('z / L'); axes[1].set_ylabel('Temperature [K]'); axes[1].grid(True)

    sns.scatterplot(data=df, x='z_over_L', y='h2o_mixing_ratio', hue='regime', palette=cluster_palette, ax=axes[2], s=30)
    axes[1].axvline(0, color='gray', linestyle='--')
    axes[1].set_title('Thermodynamic Structure: Temperature vs z/L')
    axes[1].set_xlabel('z / L'); axes[1].set_ylabel('Temperature [K]'); axes[1].grid(True)


    plt.tight_layout()
    plt.show()

    # ------------------------------
    # 2️⃣ Regime frequencies (3-row panel)
    # ------------------------------
    fig, axes = plt.subplots(3, 1, figsize=(8, 12))

    # Total count
    sns.countplot(data=df, x='regime', palette=cluster_palette, order=cluster_ids, ax=axes[0])
    axes[0].set_title('Total Regime Count (%)')
    axes[0].set_ylabel('Count'); axes[0].grid(True)

    # Hourly proportion
    diurnal = df.groupby(['hour', 'regime']).size().unstack(fill_value=0)
    diurnal = diurnal.div(diurnal.sum(axis=1), axis=0)
    diurnal.plot(ax=axes[1], color=[cluster_palette[c] for c in cluster_ids], marker='o')
    axes[1].set_title('Hourly Regime Proportion')
    axes[1].set_ylabel('Proportion'); axes[1].grid(True)

    # Monthly proportion
    monthly = df.groupby(['month', 'regime']).size().unstack(fill_value=0)
    monthly = monthly.div(monthly.sum(axis=1), axis=0)
    monthly.plot(ax=axes[2], color=[cluster_palette[c] for c in cluster_ids], marker='s')
    axes[2].set_title('Monthly Regime Proportion')
    axes[2].set_ylabel('Proportion'); axes[2].grid(True)

    plt.tight_layout()
    plt.show()

    # ------------------------------
    # 3️⃣ Violin plots of fluxes (after outlier removal)
    # ------------------------------
    flux_vars = ['H', 'LE', 'Tau', 'co2_flux']
    df_flux = remove_outliers_iqr(df, flux_vars)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    for i, var in enumerate(flux_vars):
        ax = axes[i//2][i%2]
        sns.violinplot(data=df_flux, x='regime', y=var, palette=cluster_palette, order=cluster_ids, ax=ax, inner='box')
        ax.set_title(f"{var} by Regime"); ax.grid(True)
    plt.tight_layout(); plt.show()

    # ------------------------------
    # 4️⃣ Weekday vs Weekend comparison (2x2 panel)
    # ------------------------------
    df['day_type'] = np.where(df['is_weekend'], 'Weekend', 'Weekday')
    stats = []
    for var in flux_vars:
        for regime in cluster_ids:
            sub = df[df['regime'] == regime]
            wk = sub[sub['day_type'] == 'Weekday'][var].dropna()
            we = sub[sub['day_type'] == 'Weekend'][var].dropna()
            if len(wk) > 5 and len(we) > 5:
                stat, p = mannwhitneyu(wk, we, alternative='two-sided')
                stats.append({'Regime': regime, 'Flux': var, 'p-value': p,
                              'Weekday Mean': wk.mean(), 'Weekend Mean': we.mean()})
    stat_df = pd.DataFrame(stats)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    for i, var in enumerate(flux_vars):
        ax = axes[i//2][i%2]
        subset = stat_df[stat_df['Flux'] == var]
        melted = pd.melt(subset, id_vars=['Regime', 'p-value'], value_vars=['Weekday Mean', 'Weekend Mean'],
                         var_name='Day Type', value_name='Mean')
        melted['Day Type'] = melted['Day Type'].str.replace(" Mean", "")
        sns.barplot(data=melted, x='Regime', y='Mean', hue='Day Type', hue_order=['Weekday', 'Weekend'],
                    palette={'Weekday': '#4daf4a', 'Weekend': '#984ea3'}, ax=ax)
        for _, row in subset.iterrows():
            if row['p-value'] < 0.05:
                xpos = cluster_ids.index(row['Regime'])
                ymax = max(row['Weekday Mean'], row['Weekend Mean'])
                ax.text(xpos, ymax * 1.05, '*', ha='center', color='red', fontsize=14)
        ax.set_title(f"{var}: Weekday vs Weekend")
        ax.grid(True)
    plt.tight_layout(); plt.show()

    return df


In [3]:

# ----------------------------------------
# 🚀 RUN EVERYTHING
# ----------------------------------------

# Path to NetCDF files
data_path = "/Users/bhupendra/projects/crocus/data/flux_data/data/netcdf/resnc/"

# Load and preprocess
df = load_all_variables(data_path)
df = add_temporal_columns(df)
df = assign_season(df)

# Apply QC
qc_mapping = {
    'H': 'qc_H', 'LE': 'qc_LE',
    'co2_flux': 'qc_co2_flux', 'h2o_flux': 'qc_h2o_flux',
    'Tau': 'qc_Tau'
}
df = clean_by_qc_flags(df, qc_mapping)

# Stability filtering
df = compute_stability(df)


In [1]:

# Final clustering variables
cluster_vars=['air_temperature', 'u*', 'z_over_L']

# Drop outliers
df = remove_outliers_iqr(df, cluster_vars)

# Run clustering + analysis
df_clustered = run_clustering_and_analysis(df, cluster_vars=cluster_vars, nclust=5)


NameError: name 'remove_outliers_iqr' is not defined